In [1]:
from dotenv import load_dotenv

from finetuner.dataset import FileDataset
from finetuner.finetuner import Finetuner
from finetuner import Client

load_dotenv()
client = Client.for_anyscale()

dataset = FileDataset.from_file("search_classifier.json")
finetuner = Finetuner(client=client)

In [20]:
file_id = finetuner.upload_dataset(dataset)

Job status: pending


In [ ]:
if file_id:
    job = finetuner.start_job("meta-llama/Llama-2-7b-chat-hf", dataset)